## Quick tour of SOFA and sofar

If you are new to SOFA and/or sofar, this is a good place to start. SOFA is short for *Spatially Oriented Format for Acoustics* and is an open file format for saving acoustic data, as for example head-related impulse responses (HRIRs). Good places to get more information about SOFA after this introduction are

* The [SOFA paper](https://doi.org/10.17743/jaes.2022.0026)
* The documentation of the [SOFA conventions](https://sofar.readthedocs.io/en/stable/resources/conventions.html)
* [sofaconventions.org](https://www.sofaconventions.org)
* The SOFA standard [AES69-2022](https://www.aes.org/publications/standards/search.cfm?docID=99)


### Creating SOFA objects

To cover a variety of data, SOFA offers different *conventions*. A convention defines, what data can be saved and how it is saved. You should always find the most specific convention for your data. This will help you to identify relevant data and meta data that you should provide along the actual acoustic data. Using sofar, a list of possible conventions can be obtained with

In [ ]:
import sofar as sf
import os

sf.list_conventions()

Let us assume, that you want to store head-related impulse responses (HRIRs). In this case the most specific convention is *SimpleFreeFieldHRIR*. To create a SOFA object use

In [ ]:
sofa = sf.Sofa('SimpleFreeFieldHRIR')
print(sofa)

The return value `sofa` is a `sofar.Sofa` object filled with the default values of the `SimpleFreeFieldHRIR` convention. Note that you can also get a sofa object that has only the mandatory attributes by calling `sf.Sofa('SimpleFreeFieldHRIR', mandatory=True)`. However, it is recommended to start with all attributes and discard empty optional attributes before saving the data.

### Getting information about SOFA objects

An formal description of each convention is given in the [sofar documentation](https://sofar.readthedocs.io/en/stable/resources/conventions.html). You might have noted from the documentation that three different kinds of data types can be stored in SOFA files:

* **Attributes:**
    Attributes are meta data stored as strings. There are two kinds of attributes. Global attributes give information about the entire data stored in a SOFA file. All entires starting with *GLOBAL* are such attributes. Specific attributes hold meta data for a certain variable. These attributes thus start with the name of the variable followed by an underscore, e.g., *ListenerPosition_Units*. An exception to this rule are the data variables, e.g, *Data_IR* is not an attribute but a double variable.
* **Double Variables:**
    Variables of type *double* store numeric data and can be entered as numbers, lists, or numpy arrays.
* **String Variables:**
    Variables of type *string* store strings and can be entered as strings, lists of string, or numpy string arrays.

The data can be mandatory, optional, and read only and must have a shape (dimension in SOFA language) according to the underlying convention. Read on for more information.

To get a quick insight into SOFA objects

* ``sofa.inspect`` prints the data stored in a SOFA object or at least gives
  the shape in case of large arrays that would clutter the output. This is
  helpful when reading data from an existing SOFA object.
* ``sofa.list_dimensions`` prints the dimensions of the data inside the SOFA
  object.
* ``sofa.get_dimension`` returns the size of a specific dimension.

For the *SimpleFreeFieldHRIR* SOFA object we have the following dimensions

In [ ]:
sofa.list_dimensions

In this case, `M` denotes the number of source positions for which HRIRs are available, `R` is the number of ears - which is two - and `N` gives the lengths of the HRIRs in samples. `S` is zero, because the convention does not have any string variables. `C` is always three, because coordinates are either given by x, y, and z values or by their azimuth, elevation and radius in degree.

It is important to be aware of the dimensions and enter data as determined by the convention. SOFA sets the `dimensions` implicitly. This means the dimensions are derived from the data itself, as indicated by the output of `sofa.list_dimensions` above (*set by...*). In some cases, variables can have different shapes. An example for this is the `ReceiverPosition` which can be of shape RCI or RCM. To get a dimension as a variable use

In [ ]:
sofa.get_dimension('N')

Let's assume you want to have a better overview of the data contained in the Sofa object. For this purpose you can use `sofa.inspect`. This lists the information contained in the SOFA attributes, and gives at least the shapes of all SOFA variables. In case a variable contains only a few entries, its values will be shown as well. This will most likely give you a better idea of the data then
looking at the definition of the convention or calling `sofa.list_dimensions`.

In [ ]:
sofa.inspect()

### Adding data to SOFA objects

Data can be obtained by simply calling attributes of the Sofa object, e.g.

In [ ]:
sofa.Data_IR

Note that all variables are stored as [numpy arrays](https://numpy.org/doc/stable/reference/generated/numpy.array.html), which means that you can use [indexing, and slicing](https://numpy.org/doc/stable/user/basics.indexing.html) just like with any other numpy array. For example you can get only the left ear data with

In [ ]:
sofa.Data_IR[:, 0]

Data can be added in the same way. The HRIRs and source position can for example be set with

In [ ]:
sofa.Data_IR = [[[1, 0, 0], [0, 0.5, 0]]]
sofa.SourcePosition = [90, 0, 1.5]

Now, the SOFA object contains a single HRIR - which is `[1, 0, 0]` for the left ear and `[0, 0.5, 0]` for the right ear - for a source at `90` degree azimuth, `0` degree elevation and a radius of `1.5` meter. Note that we entered lists and that sofar automatically converts the lists to numpy arrays. Sofar handles this in two steps:

1. When entering data as lists it is converted to a numpy array with at least two dimensions.
2. Missing dimensions are appended when writing the SOFA object to disk.

You should now fill all mandatory entries of the SOFA object if you were for real. For this example we'll cut it here for the sake of brevity. Let us, however, delete an optional entry that we do not need at this point

In [ ]:
sofa.delete('SourceUp')

In some cases you might want to add custom data. Although third party applications most likely won't make use of non-standardized data this can be useful for documentation and research use. Try this
to add a temperature value and unit. Note that you have to specify the data type and shape (dimensions) if you are adding SOFA variables

In [ ]:
sofa.add_variable('Temperature', 25.1, 'double', 'MI')
sofa.add_attribute('Temperature_Units', 'degree celsius')

After entering the data, the SOFA object should be verified to make sure that your data agrees with the SOFA standard and that if can be read by other applications.

In [ ]:
sofa.verify()

This will check specific rules determined by the SOFA standard AES69 and general rules such as:

- Are all mandatory data contained?
- Are the names of variables and attributes in accordance with the SOFA
  standard?
- Are the data types in accordance with the SOFA standard?
- Are the dimensions of the variables consistent and in accordance
  to the SOFA standard?
- Are the values of attributes consistent and in accordance to the
  SOFA standard?

If any violations are detected, an error is raised.

### Reading and writing SOFA objects

Note that you usually do not need to call ``sofa.verify()`` separately  because it is by default called if you create, write, or read a SOFA object. To write your SOFA object to disk type

In [ ]:
sf.write_sofa(os.path.join('data', 'my_first.sofa'), sofa)

It is good to know that SOFA files are essentially netCDF4 files which is
based on HDF5. They can thus be viewed with [HDF View](https://www.hdfgroup.org/downloads/hdfview/).

To read your sofa file you can use

In [ ]:
sofa_read = sf.read_sofa(os.path.join('data', 'my_first.sofa'))

And to see that the written and read files contain the same data you can check

In [ ]:
sf.equals(sofa, sofa_read)

### Upgrading SOFA files

SOFA conventions might get updates to fix bugs in the conventions, in case new conventions are introduced, or in case conventions get deprecated. To find out if SOFA data from a file is up to data load it and call to get a list upgrade choices or let you know that the convention is already up
to date.

In [ ]:
sofa.upgrade_convention()

### Next steps

For detailed information about sofar refer to the [sofar documentation](https://sofar.readthedocs.io). For examples on how to work with the data inside SOFA files refer to :ref:`working_with_sofa`.